## Segment central zone and periportal zone: Defined by presence of Glul (ch1) and Ecadh (ch2) resp.

### Load libraries, define paths and parameters

In [3]:
# Libraries
import spatialdata as sd
import os
import yaml
import numpy as np
import dask.array as da
from spatialdata.models import  Labels2DModel
from functions.zonation_segmentation_utils import *
import cv2

In [4]:
#Paths
sample_type = 'Flox_Control'
data_dir = '/srv/data/michielvc/data/CSF1_ligand_data/SpatialData/'

# Open config file and extract relevant information
config = yaml.safe_load(open(data_dir + f'{sample_type}_config.yaml'))
file_names = config['file_names']

# Parameters
glul_ch = config['marker_channel']['Glul']
ecadh_ch = config['marker_channel']['Ecadh']

central_zone_label = config['zonation_labels']['central_zone']
periportal_zone_label = config['zonation_labels']['periportal_zone']

central_quantile_threshold = 85

morph_structuring_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30,30)).astype(np.uint8)

### Load SpatialData object

In [5]:
sdata = sd.read_zarr(os.path.join(data_dir, f'{sample_type}.sd'))
sdata

SpatialData object with:
├── Images
│     ├── 'E2M1FloxControl_preprocessed': SpatialImage[cyx] (5, 7533, 16659)
│     ├── 'E2M1FloxControl_raw': SpatialImage[cyx] (5, 7533, 16659)
│     ├── 'E2M3FloxControl_preprocessed': SpatialImage[cyx] (5, 11178, 18468)
│     ├── 'E2M3FloxControl_raw': SpatialImage[cyx] (5, 11178, 18468)
│     ├── 'E2M5FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 12987)
│     ├── 'E2M5FloxControl_raw': SpatialImage[cyx] (5, 9342, 12987)
│     ├── 'E2M6FloxControl_preprocessed': SpatialImage[cyx] (5, 11205, 14823)
│     ├── 'E2M6FloxControl_raw': SpatialImage[cyx] (5, 11205, 14823)
│     ├── 'E2M7FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 16632)
│     ├── 'E2M7FloxControl_raw': SpatialImage[cyx] (5, 9342, 16632)
│     ├── 'E2M8FloxControl_preprocessed': SpatialImage[cyx] (5, 7533, 14823)
│     ├── 'E2M8FloxControl_raw': SpatialImage[cyx] (5, 7533, 14823)
│     ├── 'E2M15FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 14823)
│     ├── 'E

### Get segment periportal_zones and central_zones from glul and ecadh channel resp. 

In [4]:
# Get data
for file_name in file_names:
    image = sdata.images[file_name + '_preprocessed'].data

    # Get channels of interest
    glul_channel = image[glul_ch]
    ecadh_channel = image[ecadh_ch]

    # Make zonation mask container
    zonation_mask = da.zeros_like(ecadh_channel, dtype=np.uint8)

    # Get central zone and add to container with right index
    central_zone = segment_central_zones(glul_channel=glul_channel, structuring_element=morph_structuring_kernel) # reads full image!! TODO
    zonation_mask += (central_zone * central_zone_label).astype(np.uint8)

    #get periportal zone and add to container with right index
    periportal_zone = segment_periportal_zones(ecadh_channel=ecadh_channel, structuring_element=morph_structuring_kernel) # reads full image!! TODO
    zonation_mask += (periportal_zone * periportal_zone_label).astype(np.uint8)

    # remove duplicate regions (central + portal = 3)
    zonation_mask[zonation_mask > 2] = 0
    try:
        sdata.add_labels(name= file_name + '_zonation', labels=Labels2DModel.parse(zonation_mask), overwrite=True)
    except:
        rechuncked_zonation_mask = da.asarray(zonation_mask).rechunk() # I don't understand why this is necessary, but it circuvents an error sometimes
        sdata.add_labels(name= file_name + '_zonation', labels=Labels2DModel.parse(rechuncked_zonation_mask), overwrite=True)


INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


no parent found for <ome_zarr.reader.Label object at 0x7f516dfe3e20>: None


INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


no parent found for <ome_zarr.reader.Label object at 0x7f516df2e670>: None


INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


...

no parent found for <ome_zarr.reader.Label object at 0x7f516dabc6a0>: None


INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


no parent found for <ome_zarr.reader.Label object at 0x7f516dc234f0>: None


INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


no parent found for <ome_zarr.reader.Label object at 0x7f516da42790>: None
